# Data Cleaning
***

## Data Aquisition

In [1]:
import pandas as pd
import numpy as np
from numba import njit, jit
from typing import TypeVar
import multiprocessing
from joblib import Parallel, delayed
from IPython.core.display import HTML, display
import time
import datetime
from scipy import stats
import os
import warnings

warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 20)

num_cores = multiprocessing.cpu_count()
PandasDataFrame = TypeVar('pandas.core.frame.DataFrame')
NaN = np.nan
highlighted_countries = ["US", "Australia", "Canada", "China", "UK"]

### Time Series Tables

These are the U.S. and Global time series tables from the CSSE database

In [2]:
fips = pd.read_csv('../data/population/UID_ISO_FIPS_LookUp_Table.csv')
fips_US = fips[777:] # only include county specific
fips_US = fips_US[fips['Country_Region'] == 'US']
fips_US = fips_US[['FIPS', 'Admin2', 'Province_State', 'Country_Region']]
fips_US = fips_US.rename({'Admin2': 'County'},axis=1)
fips_US['FIPS'] = fips_US['FIPS'].astype('str').str.slice(0,-2)

cases_US = pd.read_csv("../data/pandemic/time_series_covid19_confirmed_US.csv")

cases_US = cases_US[5:] # exclude US territories

cases_US = cases_US.drop(["FIPS","Combined_Key","code3","iso2", "iso3","UID"], axis=1)
cases_US = cases_US.rename(columns={"Admin2": "County", "Long_": "Long"})

cases_global = pd.read_csv("../data/pandemic/time_series_covid19_confirmed_global.csv")
cases_global = cases_global.rename(columns={"Province/State": "Province_State", 
                                            "Country/Region": "Country_Region"})
cases_global["County"] = NaN

In [3]:
display(HTML("<p style='font-size: 15px'>Global Cases</p>"))
cases_global

,Province_State,Country_Region,Lat,Long,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,...,6/22/2020,6/23/2020,6/24/2020,6/25/2020,6/26/2020,6/27/2020,6/28/2020,6/29/2020,6/30/2020,County
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,29157,29481,29640,30175,30451,30616,30967,31238,31517,NaN
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,1995,2047,2114,2192,2269,2330,2402,2466,2535,NaN
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,11920,12076,12248,12445,12685,12968,13273,13571,13907,NaN
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,NaN
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,186,189,197,212,212,259,267,276,284,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,...,702,707,710,711,712,713,713,713,714,NaN
262,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,967,992,1015,1076,1089,1103,1118,1128,1158,NaN
263,NaN,Comoros,-11.645500,43.333300,0,0,0,0,0,0,...,247,265,265,272,272,272,272,272,303,NaN
264,NaN,Tajikistan,38.861034,71.276093,0,0,0,0,0,0,...,5513,5567,5630,5691,5747,5799,5849,5900,5900,NaN


In [4]:
display(HTML("<p style='font-size: 15px'>U.S. Cases</p>"))
cases_US

,County,Province_State,Country_Region,Lat,Long,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,...,6/21/2020,6/22/2020,6/23/2020,6/24/2020,6/25/2020,6/26/2020,6/27/2020,6/28/2020,6/29/2020,6/30/2020
5,Autauga,Alabama,US,32.539527,-86.644082,0,0,0,0,0,...,428,436,447,463,473,482,492,497,521,530
6,Baldwin,Alabama,US,30.727750,-87.722071,0,0,0,0,0,...,415,422,435,449,462,500,539,559,626,663
7,Barbour,Alabama,US,31.868263,-85.387129,0,0,0,0,0,...,271,276,279,287,303,309,314,314,319,322
8,Bibb,Alabama,US,32.996421,-87.125115,0,0,0,0,0,...,124,126,132,138,146,150,158,159,162,167
9,Blount,Alabama,US,33.982109,-86.567906,0,0,0,0,0,...,146,150,156,165,173,181,185,186,196,204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3256,Central Utah,Utah,US,39.372319,-111.575868,0,0,0,0,0,...,102,110,114,117,117,127,134,143,159,169
3257,Southeast Utah,Utah,US,38.996171,-110.701396,0,0,0,0,0,...,31,31,31,32,32,33,34,35,35,36
3258,Southwest Utah,Utah,US,37.854472,-111.441876,0,0,0,0,0,...,1056,1104,1151,1200,1258,1302,1361,1428,1467,1519
3259,TriCounty,Utah,US,40.124915,-109.517442,0,0,0,0,0,...,38,39,39,40,42,45,46,48,48,50


#### Expand Time Series Data

**Here I wanted to treat the time series data as one feature rather than separate variabels. I explored several ways to approach this, but with a lack of user friendly solutions, I iteratively expanded each row. I was able to minimize the runtime through multiprocessing.**

**The global table was filtered by the key countries of interest: U.S., Australia, Canada, China, and UK. I chose these countries as they have province specific data making them comparable at a detailed level. The U.S. cases were kept in a separate table at the county level for separate analysis, but were aggregated to the state level within the global table.**

In [ ]:
cases_total_temp = pd.concat([cases_US, cases_global], sort=False)
cases_total_temp = cases_total_temp[cases_total_temp['Country_Region'].isin(highlighted_countries)]
cases_total_temp = cases_total_temp[cases_total_temp['Province_State'].notnull()]

In [ ]:
def get_rows(row):
    temp = pd.DataFrame(columns=pd.DataFrame(columns=['County','Province_State','Country_Region', 
                                                      'Lat','Long','Date','Total_Cases']))
    for date in row[5:].iteritems():
            new_row = row[:5]
            new_row["Date"] = date[0]
            new_row["Total_Cases"] = date[1]
            temp = pd.concat([temp, new_row.to_frame().transpose()])
    return temp

def convert_time_series():
    cols = cases_total_temp.columns[:5].append(pd.Index(["Date","Total_Cases"]))
    temp = pd.DataFrame(columns=cols)

    row_n = 0
    result = Parallel(n_jobs=num_cores-1)(delayed(get_rows)(j) for i, j in cases_total_temp.iterrows())
    return pd.concat(result)

start_time = time.time()
cases_total = convert_time_series()
end_time = time.time() - start_time
# print("--- %s seconds ---" % (end_time))

In [ ]:
# save data in case a kernal restart is required
cases_total[['Lat', 'Long']] = cases_total[['Lat', 'Long']].apply(pd.to_numeric)
cases_total['Date'] = pd.to_datetime(cases_total['Date'], cache=True)
cases_total.to_csv("../data/pandemic/covid_19_time_series_all.csv")

In [5]:
# load data if kernal is restarted
cases_total = pd.read_csv("../data/pandemic/covid_19_time_series_all.csv",low_memory=False).iloc[:,1:]
cases_total['Date'] = pd.to_datetime(cases_total['Date'], cache=True)

In [6]:
cases_US = cases_total[cases_total['Country_Region'] == 'US']

# aggregate US cases by state
cases_US_temp = cases_US.groupby(['Province_State','Date']).agg({
    'Country_Region': pd.Series.mode,
    'Lat': np.mean,
    'Long': np.mean,
    'Total_Cases': np.sum,
})

cases_total = cases_total.drop('County', axis=1).append(cases_US_temp.reset_index(), sort=False)

In [7]:
display(HTML("<p style='font-size: 15px'>U.S. By Counties</p>"))
cases_US

,County,Province_State,Country_Region,Lat,Long,Date,Total_Cases
0,Autauga,Alabama,US,32.539527,-86.644082,2020-01-22,0
1,Autauga,Alabama,US,32.539527,-86.644082,2020-01-23,0
2,Autauga,Alabama,US,32.539527,-86.644082,2020-01-24,0
3,Autauga,Alabama,US,32.539527,-86.644082,2020-01-25,0
4,Autauga,Alabama,US,32.539527,-86.644082,2020-01-26,0
...,...,...,...,...,...,...,...
524211,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-06-26,814
524212,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-06-27,846
524213,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-06-28,872
524214,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-06-29,919


In [8]:
display(HTML("<p style='font-size: 15px'>Global</p>"))
cases_total

,Province_State,Country_Region,Lat,Long,Date,Total_Cases
0,Alabama,US,32.539527,-86.644082,2020-01-22,0
1,Alabama,US,32.539527,-86.644082,2020-01-23,0
2,Alabama,US,32.539527,-86.644082,2020-01-24,0
3,Alabama,US,32.539527,-86.644082,2020-01-25,0
4,Alabama,US,32.539527,-86.644082,2020-01-26,0
...,...,...,...,...,...,...
8528,Wyoming,US,39.639935,-98.580983,2020-06-26,1368
8529,Wyoming,US,39.639935,-98.580983,2020-06-27,1392
8530,Wyoming,US,39.639935,-98.580983,2020-06-28,1417
8531,Wyoming,US,39.639935,-98.580983,2020-06-29,1450


### Adding Temperature, Political Party, Protest, and Lockdown Variables

**After organizing the pandemic data, I added a series of other interesting variables mainly to the U.S. dataset including average temperature per month, political party controlling the current local governemnt, a count for the number of recoreded Black Lives Matter protests in the area, COVID-19 lockdown phases and restriction rating from the Washington Post.**

In [9]:
### COVID-19 Phases ###

In [10]:
phases = pd.read_csv("../data/pandemic/WP_phases_6.30.20.csv")
for i in range(4):
    phases[f'Phase.{i}'] = pd.to_datetime(phases[f'Phase.{i}'], cache=True)

cases_US = cases_US.merge(phases, left_on='Province_State', right_on='State')

In [11]:
def set_phases(row):
    if  row['Date'] < row['Phase.0']:
        return -1
    elif row['Date'] >= row['Phase.0'] and row['Date'] < row['Phase.1']:
        return 0
    elif row['Date'] >= row['Phase.1'] and row['Date'] < row['Phase.2']:
        return 1
    elif row['Date'] >= row['Phase.2'] and row['Date'] < row['Phase.3']:
        return 2
    else:
        return 3

cases_US['Current_Phase'] = pd.Categorical(cases_US.apply(set_phases, axis=1))

In [12]:

# cases_US[
#     (cases_US['Date'] < (cases_US['Date'][10] + pd.DateOffset(days=100))) & 
#     (cases_US['County'] == cases_US['County'][10]) & 
#     (cases_US['Province_State'] == cases_US['Province_State'][10])]['Total_Cases'].max()


In [13]:
### Temperature Data ###

In [14]:
fips_US['County_FIPS'] = fips_US['FIPS'].str.slice(-3)
cases_US = cases_US.merge(fips_US)

In [15]:
cases_US['Month'] = cases_US['Date'].dt.to_period('M')

files = []
for r, d, f in os.walk("../data/temperature/"):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(r, file))

dates = pd.date_range('2020-01-31', periods=len(files), freq='M')
temps = pd.DataFrame(columns=pd.DataFrame(columns=["Location ID","Location","Value","Rank",
                                                   "Anomaly (1901-2000 base period)",
                                                   "1901-2000 Mean", "Month"]))
for i, file in enumerate(files):
    temp = pd.read_csv(file)
    temp['Month'] = dates[i]
    temps = pd.concat([temps, temp])
temps['Month'] = temps['Month'].dt.to_period('M')

temps['Abbreviation'] = temps['Location ID'].str.slice(0,2)
temps['County_FIPS'] = temps['Location ID'].str.slice(-3)
temps = temps.drop(['Location ID', 'Location', 'Rank', 'Anomaly (1901-2000 base period)', '1901-2000 Mean'], axis=1)
temps = temps.rename({'Value': 'Avg_Temp'}, axis=1)

temps_US = pd.merge(cases_US, temps, how='left', on=['Abbreviation','Month', 'County_FIPS'])
temps_US = temps_US.drop('Province_State', axis=1)

In [16]:
# fill in missing temps with state average by month
avg_m_temps_us = (temps_US.groupby(['State','Month'])
    .mean()
    .reset_index()
    .drop(['Lat','Long','Total_Cases'], axis=1))
rows = temps_US[np.isnan(temps_US['Avg_Temp'])][['County', 'State', 'Date', 'FIPS', 'Month']]

missing_temps = rows.reset_index().merge(avg_m_temps_us, on=['State', 'Month']).set_index('index')
missing_temps['Avg_Temp'] = round(missing_temps['Avg_Temp'], ndigits=1)

temps_US.loc[missing_temps.index,['Avg_Temp']] = missing_temps['Avg_Temp']
cases_US = temps_US.drop('County_FIPS', axis=1)

In [17]:
### Protest Data ###

In [18]:
BLM_global = pd.read_csv('../data/protests/BLMProtestsCount.csv').iloc[:,1:]
BLM_global = BLM_global.rename({'Value.properties.Country': 'Country_Region',
                                'Region': 'Province_State', 'Count': 'Protest_Count',
                                'State': 'Abbreviation'}, axis=1)

# Data Cleaning
BLM_global = BLM_global[BLM_global['Province_State'].notnull()] # only compare region specific
BLM_global.loc[BLM_global['Country_Region'] == 'USA', 'Country_Region'] = 'US'
BLM_global['Province_State'] = BLM_global['Province_State'].str.replace(' County', '')
BLM_global['Province_State'] = BLM_global['Province_State'].str.replace(' Parish', '')
BLM_global.loc[BLM_global['Province_State'] == 'Baltimore city', 'Province_State'] = 'Baltimore City'
BLM_global = BLM_global[BLM_global['Country_Region'].isin(highlighted_countries)]

BLM_US = BLM_global[BLM_global['Country_Region'] == 'US']
BLM_US = BLM_US.rename({'Province_State': 'County', 'State': 'Abbreviation'}, axis=1)

# US Protests
temp = pd.merge(cases_US, BLM_US, how='left')
temp.loc[temp['Protest_Count'].isnull(), 'Protest_Count'] = 0
temp['Protest_Count'] = temp['Protest_Count'].astype('int32')
cases_US = temp

# Generalize Gobal by Province/Region
States = cases_US[['State', 'Abbreviation']].groupby(["State", "Abbreviation"]).max().reset_index()

BLM_US = BLM_US.merge(States)
BLM_US = BLM_US.rename({'State': 'Province_State'},axis=1).drop('County', axis=1)
BLM_US = BLM_US.groupby(['Province_State', 'Country_Region', 'Abbreviation']).sum().reset_index()

BLM_total = BLM_global[BLM_global['Country_Region'] != 'US']
BLM_total = pd.concat([BLM_US, BLM_total]).reset_index().drop('index', axis=1)
cases_total = cases_total.merge(BLM_total, how='left')

In [19]:
display(HTML("<p style='font-size: 15px'>U.S. By Counties</p>"))
cases_US

,County,Country_Region,Lat,Long,Date,Total_Cases,State,Abbreviation,Phase.0,Phase.1,Phase.2,Phase.3,Restriction Rating,Governer.Party,Current_Phase,FIPS,Month,Avg_Temp,Protest_Count
0,Autauga,US,32.539527,-86.644082,2020-01-22,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,1001,2020-01,50.5,0
1,Autauga,US,32.539527,-86.644082,2020-01-23,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,1001,2020-01,50.5,0
2,Autauga,US,32.539527,-86.644082,2020-01-24,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,1001,2020-01,50.5,0
3,Autauga,US,32.539527,-86.644082,2020-01-25,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,1001,2020-01,50.5,0
4,Autauga,US,32.539527,-86.644082,2020-01-26,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,1001,2020-01,50.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521635,Unassigned,US,0.000000,0.000000,2020-06-26,0,Wyoming,WY,2020-03-15,2020-03-15,2020-05-01,2020-05-15,minor,R,3,90056,2020-06,NaN,0
521636,Unassigned,US,0.000000,0.000000,2020-06-27,0,Wyoming,WY,2020-03-15,2020-03-15,2020-05-01,2020-05-15,minor,R,3,90056,2020-06,NaN,0
521637,Unassigned,US,0.000000,0.000000,2020-06-28,0,Wyoming,WY,2020-03-15,2020-03-15,2020-05-01,2020-05-15,minor,R,3,90056,2020-06,NaN,0
521638,Unassigned,US,0.000000,0.000000,2020-06-29,0,Wyoming,WY,2020-03-15,2020-03-15,2020-05-01,2020-05-15,minor,R,3,90056,2020-06,NaN,0


In [20]:
display(HTML("<p style='font-size: 15px'>Global</p>"))
cases_total

,Province_State,Country_Region,Lat,Long,Date,Total_Cases,Abbreviation,Protest_Count
0,Alabama,US,32.539527,-86.644082,2020-01-22,0,AL,38.0
1,Alabama,US,32.539527,-86.644082,2020-01-23,0,AL,38.0
2,Alabama,US,32.539527,-86.644082,2020-01-24,0,AL,38.0
3,Alabama,US,32.539527,-86.644082,2020-01-25,0,AL,38.0
4,Alabama,US,32.539527,-86.644082,2020-01-26,0,AL,38.0
...,...,...,...,...,...,...,...,...
541599,Wyoming,US,39.639935,-98.580983,2020-06-26,1368,WY,14.0
541600,Wyoming,US,39.639935,-98.580983,2020-06-27,1392,WY,14.0
541601,Wyoming,US,39.639935,-98.580983,2020-06-28,1417,WY,14.0
541602,Wyoming,US,39.639935,-98.580983,2020-06-29,1450,WY,14.0


### Population and Demographics Variables (US)

**Lastly, I added American Community Survey (ACS) estimates from the Census Bureau including demographic data at the county level in the U.S along with the population density of each county.**

In [21]:
### Popualtion Data ###

In [22]:
pop_US = pd.read_csv('../data/population/county_area_2010.csv')
pop_US = pop_US[['fips', 'PST045214', 'AGE775213', 'RHI125213',
                 'RHI225213', 'RHI325213', 'RHI425213', 'RHI525213',
                 'RHI625213', 'RHI725213', 'POP645213', 'HSD310213',
                 'LND110210', 'POP060210']]
pop_US = pop_US.rename({'fips': 'FIPS', 'PST045214': 'POP.2014', 'AGE775213': 'Perc.Over.65',
                        'RHI125213': 'Perc.White','RHI225213': 'Perc.Black', 
                        'RHI325213': 'Perc.Native', 'RHI425213': 'Perc.Asian',
                        'RHI525213': 'Perc.Pac.Island', 'RHI625213': 'Perc.Mixed',
                        'RHI725213': 'Perc.His.Lat', 'POP645213': 'Perc.Foreign.Born', 
                        'HSD310213': 'Avg.Person.Per.Household',
                        'LND110210': 'Land.Area.sq.mi', 'POP060210': 'POP.per.sq.mi'},axis=1)
pop_US['FIPS'] = pop_US['FIPS'].astype('str')

In [23]:
# temp = pop_US.merge(cases_US, how='right', on='FIPS')
# cases_US = temp.dropna()

pop_global = pd.read_csv("../data/population/WPP2019_TotalPopulationBySex.csv")
pop_global = pop_global[pop_global['Time'] == 2020]
pop_global = pop_global.groupby('Location').mean().reset_index()
pop_global = pop_global.drop(['VarID', 'Time', 'MidPeriod'], axis=1)
pop_global = pop_global.rename({'LocID': 'FIPS', 'Location': 'Country'},axis=1) # technically incorrect but makes joining simpler
pop_global['FIPS'] = pop_global['FIPS'].astype('str').str.slice(0,-2)

In [24]:
display(HTML("<p style='font-size: 15px'>U.S. By Counties</p>"))
pop_US

,FIPS,POP.2014,Perc.Over.65,Perc.White,Perc.Black,Perc.Native,Perc.Asian,Perc.Pac.Island,Perc.Mixed,Perc.His.Lat,Perc.Foreign.Born,Avg.Person.Per.Household,Land.Area.sq.mi,POP.per.sq.mi
0,0,318857056,14.1,77.7,13.2,1.2,5.3,0.2,2.4,17.1,12.9,2.63,3531905.43,87.4
1,1000,4849377,14.9,69.8,26.6,0.7,1.3,0.1,1.5,4.1,3.5,2.55,50645.33,94.4
2,1001,55395,13.5,78.1,18.4,0.5,1.1,0.1,1.7,2.7,1.6,2.71,594.44,91.8
3,1003,200111,18.1,87.3,9.5,0.7,0.9,0.1,1.5,4.6,3.6,2.52,1589.78,114.6
4,1005,26887,15.9,50.2,47.6,0.6,0.5,0.2,0.9,4.8,2.9,2.66,884.88,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,45010,9.0,94.2,1.7,1.2,0.9,0.2,1.7,15.9,5.5,2.62,10426.65,4.2
3191,56039,22930,11.8,94.6,1.5,1.0,1.3,0.1,1.5,15.1,9.1,2.76,3995.38,5.3
3192,56041,20904,10.5,95.6,0.8,1.3,0.4,0.2,1.7,9.1,3.4,2.85,2081.26,10.1
3193,56043,8322,19.2,95.0,0.8,1.7,0.7,0.1,1.7,14.2,5.0,2.40,2238.55,3.8


**For the global data, the European Union had population numbers per county. These are 2020 estimates.**

In [25]:
display(HTML("<p style='font-size: 15px'>Global</p>"))
pop_global

,Country,FIPS,PopMale,PopFemale,PopTotal,PopDensity
0,Afghanistan,4,19976.265,18952.076,38928.341,59.627
1,Africa,903,669878.340,670719.773,1340598.113,45.216
2,African Group,1823,668995.562,669831.029,1338826.591,45.571
3,African Union,1560,669307.822,670116.099,1339423.921,45.182
4,African Union: Central Africa,2080,79260.910,79358.728,158619.638,30.066
...,...,...,...,...,...,...
469,World,900,3929973.953,3864824.776,7794798.729,59.917
470,World Bank Regional Groups (developing only),1801,3299567.992,3229194.235,6528762.227,71.088
471,Yemen,887,15024.985,14800.983,29825.968,56.492
472,Zambia,894,9103.006,9280.950,18383.956,24.730
